In [1]:
!wget ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz -O /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta.gz

--2019-01-30 13:03:30--  ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz
           => ‘/Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta.gz’
Resolving ftp.uniprot.org... 141.161.180.197
Connecting to ftp.uniprot.org|141.161.180.197|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/uniprot/current_release/knowledgebase/complete ... done.
==> SIZE uniprot_sprot.fasta.gz ... 88385361
==> PASV ... done.    ==> RETR uniprot_sprot.fasta.gz ... done.
Length: 88385361 (84M) (unauthoritative)

uniprot_sprot.fasta 100%[===================>]  84.29M  6.85MB/s    in 11s     

2019-01-30 13:03:43 (7.44 MB/s) - ‘/Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta.gz’ saved [88385361]



In [2]:
!gunzip -k /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta.gz

In [3]:
!/Users/Shared/bioinformatics/ncbi-blast-2.5.0+/bin/makeblastdb \
-in /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta \
-out /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_db \
-dbtype prot



Building a new DB, current time: 01/30/2019 13:11:04
New DB name:   /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_db
New DB title:  /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 559077 sequences in 55.0913 seconds.


In [4]:
#Confirm Steven's .fa file is the same as Sean's
!diff /Volumes/web/halfshell/bu-git-repos/nb-2017/C_gigas/data/Cg_Giga_cont_AA.fa \
/Volumes/web/scaphapoda/Sean/Rhonda-2016-Oyster-Intermediates/Cg-Giga_cont_AA.fa

In [ ]:
#can sort database before aligning to change the order of entries. We don't care that much about taxanomic group, we car about protein functionality. So it doesn't matter for us if human gets called over mouse or vice-versa. So I am not worrying about changing the max_target_seqs parameter

In [5]:
!/Users/Shared/bioinformatics/ncbi-blast-2.5.0+/bin/blastp \
-query /Volumes/web/halfshell/bu-git-repos/nb-2017/C_gigas/data/Cg_Giga_cont_AA.fa \
-db /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_db \
-out /Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.tab \
-num_threads 8 \
-max_hsps 1 \
-max_target_seqs 1 \
-outfmt 6

In [1]:
!wc -l /Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.tab

   38294 /Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.tab


In [4]:
!awk '{if($1~/>/) print $1}' /Volumes/web/halfshell/bu-git-repos/nb-2017/C_gigas/data/Cg_Giga_cont_AA.fa | \
wc -l

   40751


In [8]:
#check how many query sequences are not in the Blast output
#first extract all sequence names from query file with first awk command
#sed command removes the carrot 
#compare query file sequence names to names blast output
#this awk command creates an array with the first file (blast output)
#the array creates a key out of the first field 'a[$1]' and stores a value 
#which in this case is the same as the key 'a[$1]=$1'
#;next tells it do this for each line in the first file (blast output)
#!($1 in a) means if the first field in the second file is not in the array we made
#print the first field of the second file
#The second file is denoted as '-' because we are passing it the file we made in the previous sed and awk commands
#add a column to the unmapped sequence names that specifies the sequences were unmapped

!awk '{if($1~/>/) print $1}' /Volumes/web/halfshell/bu-git-repos/nb-2017/C_gigas/data/Cg_Giga_cont_AA.fa | \
sed 's/>//g'|\
awk 'NR==FNR{a[$1]=$1;next}!($1 in a){print $1"\t""unmapped"}' \
/Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.tab - \
| tr '|' '.' > /Users/strigg/Desktop/gigas_blastP_uniprot/query_seq_names_NOTin_Blast_out


In [9]:
#count the number of sequence names in the query file that are NOT in the blast output
!wc -l /Users/strigg/Desktop/gigas_blastP_uniprot/query_seq_names_NOTin_Blast_out

    2457 /Users/strigg/Desktop/gigas_blastP_uniprot/query_seq_names_NOTin_Blast_out


In [19]:
#formatting output so that uniprot accessions are in one column
#first test to see if all uniprot db entries start with the same pattern "sp|"
#This awk command finds every line containing a '>' and prints the first three characters after the carrot
#Awk command literally: if first field contains a carrot, print a substring of the first field string, starting at the second character in the string, that is 3 characters long
# sort the list of substrings and print only unique substrings and their frequencies
#if there is only one substring, we can conclude all uniprot DB entries start with the same pattern "sp|"
!awk '{if($1~/>/)print substr($1,2,3)}' /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta | sort | uniq -c

559077 sp|


In [21]:
#Next check that there is only 1 pipe in each uniprot DB name after removing the "sp|" pattern
#we tell awk that our file is tab delimited, then print only the sequence name
#then sed to remove the 'sp|' pattern
#then sed to replace all non '|' characters with nothing and awk to count the number of characters remaining in each line
#https://unix.stackexchange.com/questions/18736/how-to-count-the-number-of-a-specific-character-in-each-line
#then sort and unique to see if there are different number of pipes in each line
!awk -F"\t" '{if($1~/>/)print $1}' \
/Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta | \
sed 's/sp\|//g' | sed 's/[^|]//g' | awk '{print length}' | \
sort | uniq -c

559077 1


In [23]:
#Next check that the uniprot DB does not contain the pattern "|m"
!grep '\|m' /Users/strigg/Desktop/gigas_blastP_uniprot/uniprot_swprot2019_01.fasta |\
head


In [24]:
#And that only the C_gigas sequence names contain "|m"
#We want to check this so that we can replace "\m" with ".m"
!grep '\|m' /Volumes/web/halfshell/bu-git-repos/nb-2017/C_gigas/data/Cg_Giga_cont_AA.fa | \
head


>CHOYP_043R.1.5|m.16874
>CHOYP_043R.5.5|m.64252
>CHOYP_14332.1.2|m.5643
>CHOYP_14332.1.2|m.5644
>CHOYP_14332.2.2|m.61737
>CHOYP_14332.2.2|m.61738
>CHOYP_1433E.1.2|m.3639
>CHOYP_1433E.1.2|m.3638
>CHOYP_1433E.2.2|m.63376
>CHOYP_1433E.2.2|m.63378


In [ ]:
# now in our BLAST output file, we can replace 'sp|' with nothing, "|m" with ".m", and then finally we can replace the only remaining pipe with tabs

In [42]:
#Because all uniprot DB entries start with the same pattern, we can replace it with nothing so that each entry starts with the uniprot accession
!sed 's/sp\|//g' /Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.tab | \
awk -F"\t" '{sub(/\|/,".",$1);print $0}'  \
| tr ' ' '\t' | tr '|' '\t' | \
head

CHOYP_043R.1.5.m.16874	Q06852	SLAP1_CLOTH	56.944	216	61	18	10	197	1388	1599	5.20e-08	55.8
CHOYP_043R.5.5.m.64252	Q06852	SLAP1_CLOTH	52.381	294	80	24	575	816	1351	1636	2.02e-16	88.2
CHOYP_14332.1.2.m.5643	Q2F637	1433Z_BOMMO	66.031	262	74	2	19	280	1	247	2.78e-119	344
CHOYP_14332.1.2.m.5644	P62325	BTG1_MOUSE	47.205	161	80	2	1	156	11	171	2.22e-47	155
CHOYP_14332.2.2.m.61737	Q2F637	1433Z_BOMMO	67.331	251	78	1	1	251	1	247	4.22e-119	342
CHOYP_14332.2.2.m.61738	Q8IDX6	RBP2A_PLAF7	27.778	108	76	1	31	136	2766	2873	5.98e-04	43.9
CHOYP_1433E.1.2.m.3639	Q9CWP8	DPOD4_MOUSE	38.235	102	61	2	30	130	7	107	9.73e-19	78.6
CHOYP_1433E.1.2.m.3638	P92177	1433E_DROME	77.692	260	53	1	1	255	1	260	1.95e-149	420
CHOYP_1433E.2.2.m.63376	P92177	1433E_DROME	77.692	260	53	1	1	255	1	260	1.95e-149	420
CHOYP_1433E.2.2.m.63378	Q6MG82	PRRT1_RAT	44.444	99	53	1	41	137	196	294	1.84e-18	82.8


In [43]:
#check end of file too
!sed 's/sp\|//g' /Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.tab | \
awk -F"\t" '{sub(/\|/,".",$1);print $0}'  \
| tr ' ' '\t'| tr '|' '\t' | tail

CYC_HORSE.	P00004	CYC_HORSE	100.000	105	0	0	1	105	1	105	3.35e-73	215
ALDOA_RABIT.	P00883	ALDOA_RABIT	100.000	360	0	0	1	360	1	360	0.0	745
LYSC_CHICK.	P00698	LYSC_CHICK	100.000	147	0	0	1	147	1	147	1.30e-105	300
MYG_HORSE.	P68082	MYG_HORSE	100.000	154	0	0	1	154	1	154	6.48e-109	309
OVAL_CHICK.	P01012	OVAL_CHICK	100.000	386	0	0	1	386	1	386	0.0	802
GFP_AEQVI.	P42212	GFP_AEQVI	100.000	238	0	0	1	238	1	238	1.26e-179	494
SRPP_HEVBR.	O82803	SRPP_HEVBR	100.000	204	0	0	1	204	1	204	2.12e-144	403
REF_HEVBR.	P15252	REF_HEVBR	100.000	138	0	0	1	138	1	138	1.14e-96	277
PLMP_GRIFR.	P81054	PLMP_GRIFR	100.000	348	0	0	1	348	1	348	0.0	715
KKA1_ECOLX	P00551	KKA1_ECOLX	100.000	271	0	0	1	271	1	271	0.0	565


In [44]:
!sed 's/sp\|//g' /Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.tab | \
awk -F"\t" '{sub(/\|/,".",$1);print $0}'  \
| tr ' ' '\t' | tr '|' '\t' \
> /Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.nopipe.tab

In [10]:
!cat /Users/strigg/Desktop/gigas_blastP_uniprot/giga-uniprot-blastP-out.nopipe.tab \
/Users/strigg/Desktop/gigas_blastP_uniprot/query_seq_names_NOTin_Blast_out \
> /Users/strigg/Desktop/gigas_blastP_uniprot/all_giga-uniprot-blastP-out.nopipe.tab

In [11]:
!wc -l /Users/strigg/Desktop/gigas_blastP_uniprot/all_giga-uniprot-blastP-out.nopipe.tab


   40751 /Users/strigg/Desktop/gigas_blastP_uniprot/all_giga-uniprot-blastP-out.nopipe.tab


In [12]:
#put place holders in fields for unmapped IDs
!awk -F"\t" '{if($2=="unmapped")print $1FS$2FS"@"FS"@"FS"@"FS"@"FS"@"FS"@"FS"@"FS"@"FS"@"FS"@"FS"@";else print $0}' \
/Users/strigg/Desktop/gigas_blastP_uniprot/all_giga-uniprot-blastP-out.nopipe.tab \
> /Users/strigg/Desktop/gigas_blastP_uniprot/all_giga-uniprot-blastP-out.nopipe.placeholder.tab

In [13]:
#combine Uniprot annotations with BLAST'd proteins
!awk -F"\t" 'NR==FNR{a[$1]=$0;next}{print $0"\t"a[$2]}' \
/Users/strigg/Desktop/gigas_blastP_uniprot/uniprot-reviewed%3Ayes.tab \
/Users/strigg/Desktop/gigas_blastP_uniprot/all_giga-uniprot-blastP-out.nopipe.placeholder.tab \
> /Users/strigg/Desktop/gigas_blastP_uniprot/all_giga-uniprot-blastP-out.nopipe.annotations.tab

In [14]:
#mv directory onto Metacarcinus on Gannet
!scp -r /Users/strigg/Desktop/gigas_blastP_uniprot/ /Volumes/web-1/metacarcinus/Cgigas/


In [15]:
ls -lh /Volumes/web-1/metacarcinus/Cgigas/

total 1916362
-rwx------  1 strigg  staff    40M Feb  4 14:46 all_giga-uniprot-blastP-out.nopipe.annotations.tab*
-rwx------  1 strigg  staff   3.4M Feb  4 14:46 all_giga-uniprot-blastP-out.nopipe.placeholder.tab*
-rwx------  1 strigg  staff   3.4M Feb  4 14:46 all_giga-uniprot-blastP-out.nopipe.tab*
-rwx------  1 strigg  staff   3.3M Feb  4 14:46 giga-uniprot-blastP-out.nopipe.tab*
-rwx------  1 strigg  staff   3.4M Feb  4 14:46 giga-uniprot-blastP-out.tab*
-rwx------  1 strigg  staff    90K Feb  4 14:46 query_seq_names_NOTin_Blast_out*
-rwx------@ 1 strigg  staff   321M Feb  5 16:27 uniprot-reviewed%3Ayes.tab*
-rwx------@ 1 strigg  staff   6.7K Feb  4 14:46 uniprot_annotations_first10.org*
-rwx------  1 strigg  staff   263M Feb  5 16:27 uniprot_swprot2019_01.fasta*
-rwx------  1 strigg  staff   102M Feb  4 14:46 uniprot_swprot2019_db.phr*
-rwx------  1 strigg  staff   4.3M Feb  4 14:46 uniprot_swprot2019_db.pin*
-rwx------  1 strigg  staff   192M Feb  5 16:27 uniprot_swprot2019_db.ps